In [57]:
import json
import random
from datetime import datetime, timedelta
from faker import Faker
import psycopg2
import uuid
import requests

In [58]:
fake = Faker()

In [59]:
def get_db_connection():
    return psycopg2.connect(
        host='localhost',
        port='5433',
        user='admin',
        password='password',
        database='university'
    )

def load_student_ids():
    student_ids = []
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT student_id FROM students")
        student_ids = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading student IDs: {e}")
    
    return student_ids

def load_courses():
    courses = []
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT title FROM courses")
        courses = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading courses: {e}")
    
    return courses

def load_majors():
    majors = []
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT name FROM departments")
        majors = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading majors: {e}")

    return majors

def load_counselor_ids():
    api_url = 'http://localhost:8002/counselors'
    response = requests.get(api_url)
    json_data = response.json()
    counselor_ids = []
    for obj in json_data:
        counselor_ids.append(obj['counselor_id'])

    return counselor_ids

In [60]:
student_ids = load_student_ids()
counselor_ids = load_counselor_ids()
courses = load_courses()
majors = load_majors()
print(len(student_ids), len(counselor_ids), len(courses), len(majors))

9000 12 1797 26


In [61]:
NUM_RECORDS = 300
START_DATE = datetime(2023, 8, 1)
END_DATE = datetime(2024, 8, 31)

In [62]:
MEETING_PURPOSES = [
    "Academic Progress Review",
    "Course Selection Assistance",
    "Major Declaration",
    "Study Plan Development",
    "Academic Probation Meeting",
    "Graduation Requirements Check",
    "Internship Application Help",
    "Career Path Guidance",
    "Transfer Credit Evaluation",
    "Mental Health Resources",
    "Accommodations Request",
    "Research Opportunity Discussion",
    "Study Abroad Planning",
    "Financial Aid Questions",
    "Academic Difficulty Support"
]

In [63]:
DESCRIPTION_TEMPLATES = [
    "I'm struggling with {course} and need help understanding how to improve my grade. My current grade is {grade}.",
    "I want to change my major and need advice on what courses I should take next semester.",
    "I need to review my graduation requirements to make sure I'm on track to graduate in {timeframe}.",
    "I've been dealing with {issue} this semester and it's affecting my academics. I need advice on how to proceed.",
    "I'm interested in studying abroad in {country} and need guidance on how this will affect my graduation timeline.",
    "I'm having trouble balancing my coursework with my job as a {job_title}. I need advice on time management.",
    "I would like to discuss potential career paths for someone with a {major} degree.",
    "I need help selecting courses for next semester that align with my interest in {interest_area}.",
    "I recently transferred from another university and want to discuss how my credits transferred over.",
    "I'm on academic probation and need to create a plan to improve my GPA this semester."
]

In [64]:
REGISTRATION_MONTHS = [4, 11]  # April and November
EXAM_MONTHS = [5, 12]          # May and December

def is_peak_season(dt):
    return dt.month in REGISTRATION_MONTHS or dt.month in EXAM_MONTHS

def generate_request_time(date):
    if date.weekday() >= 5:  # Weekend
        hour = random.choices(
            [10, 11, 12, 13, 14], weights=[1, 2, 3, 2, 1], k=1
        )[0]
    else:  # Weekday
        hour = random.choices(
            [9, 10, 11, 13, 14, 15, 16], weights=[1, 2, 3, 3, 2, 2, 1], k=1
        )[0]
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    return date.replace(hour=hour, minute=minute, second=second)

In [65]:
def weighted_dates():
    """Generate a list of weighted dates based on seasonality."""
    current = START_DATE
    date_weights = []

    while current <= END_DATE:
        weight = 1
        if current.weekday() < 5:
            weight += 2  # Weekday
        if is_peak_season(current):
            weight += 2
        date_weights.extend([current] * weight)
        current += timedelta(days=1)

    return date_weights

In [66]:
def generate_batch_requests(student_ids, batch_size=20):
    batch = []

    themes = [
        "Mid-term grade interventions",
        "First-generation student check-in",
        "Financial aid warning follow-up",
        "Academic probation meeting",
        "Graduation requirement check"
    ]
    selected_students = random.sample(student_ids, batch_size)
    
    date_pool = weighted_dates()
    
    for student_id in selected_students:
        theme = random.choice(themes)
        description = f"Required meeting for {theme.lower()}. Please schedule at your earliest convenience."
        date = random.choice(date_pool)
        timestamp = generate_request_time(date)
        
        batch.append({
            "request_id": str(uuid.uuid4()),
            "student_id": student_id,
            "purpose": theme,
            "description": description,
            "request_timestamp": timestamp.isoformat(),
            "requestor_type": "counselor"
        })
    
    return batch

In [67]:
def generate_mock_requests():
    date_pool = weighted_dates()
    data = []

    for _ in range(NUM_RECORDS):
        student_id = random.choice(student_ids)
        purpose = random.choice(MEETING_PURPOSES)
        date = random.choice(date_pool)
        timestamp = generate_request_time(date)
        
        template = random.choice(DESCRIPTION_TEMPLATES)
        description = template.format(
            course=random.choice(courses),
            grade=random.choice(["C-", "D+", "F", "B-", "C+"]),
            timeframe=random.choice(["Spring 2025", "Fall 2025", "Spring 2026"]),
            issue=random.choice(["anxiety", "family emergency", "health issues", "work conflicts"]),
            country=fake.country(),
            job_title=fake.job(),
            major=random.choice(majors),
            interest_area=random.choice(["data science", "marketing", "research", "sustainability", "healthcare"])
        )
        if random.random() < 0.3:
            description += "\n\n" + fake.paragraph()

        data.append({
            "request_id": str(uuid.uuid4()),
            "student_id": student_id,
            "purpose": purpose,
            "description": description,
            "request_timestamp": timestamp.isoformat(),
            "requestor_type": "student"
        })

        if len(data) % 20 == 0: # Every 10 requests
            batch_requests = generate_batch_requests(student_ids, batch_size=10)
            for req in batch_requests:
                data.append(req)

    return data

In [68]:
data = generate_mock_requests()
len(data)

590

In [69]:
data[:5]

[{'request_id': 'a3f487a1-dded-4cc3-a32f-91c20e98d920',
  'student_id': 'cp694',
  'purpose': 'Financial Aid Questions',
  'description': 'I recently transferred from another university and want to discuss how my credits transferred over.',
  'request_timestamp': '2024-06-26T11:12:57',
  'requestor_type': 'student'},
 {'request_id': 'e9971e4c-2e00-4381-a885-df95212d3d16',
  'student_id': 'hw967',
  'purpose': 'Financial Aid Questions',
  'description': "I'm having trouble balancing my coursework with my job as a Comptroller. I need advice on time management.",
  'request_timestamp': '2024-04-29T10:09:44',
  'requestor_type': 'student'},
 {'request_id': '1e584479-ad67-454a-afbc-5ebcf1fd282a',
  'student_id': 'hb378',
  'purpose': 'Course Selection Assistance',
  'description': "I'm on academic probation and need to create a plan to improve my GPA this semester.",
  'request_timestamp': '2024-04-17T10:51:55',
  'requestor_type': 'student'},
 {'request_id': 'd7761474-fc15-43c4-93a5-73f858

In [70]:
data_sorted = sorted(data, key=lambda d: d['request_timestamp'])

In [71]:
data_sorted[:3]

[{'request_id': '2376ccc4-ebb2-4eeb-9bc2-e09adb642170',
  'student_id': 'cp181',
  'purpose': 'Graduation Requirements Check',
  'description': 'I want to change my major and need advice on what courses I should take next semester.\n\nBoth serve despite return represent security woman. Lead behavior on fast.',
  'request_timestamp': '2023-08-01T13:54:06',
  'requestor_type': 'student'},
 {'request_id': '5d84c96c-2818-4384-9571-09db8ca7a20d',
  'student_id': 'ag375',
  'purpose': 'Course Selection Assistance',
  'description': 'I recently transferred from another university and want to discuss how my credits transferred over.\n\nStyle student now push arm have. Degree ready wonder can.',
  'request_timestamp': '2023-08-01T14:52:26',
  'requestor_type': 'student'},
 {'request_id': '791b1550-8d83-4901-a736-1dbdea0dec6e',
  'student_id': 'kb436',
  'purpose': 'Academic Difficulty Support',
  'description': 'I want to change my major and need advice on what courses I should take next semest

In [72]:
with open("/Users/ngochoang/Library/CloudStorage/GoogleDrive-nhungoc1508@gmail.com/My Drive/Graduate/Semester 2 (Spring 2025)/Big Data Management/Project/P1 - Landing zone/Data/past_meeting_requests.json", "w") as f:
    json.dump(data_sorted, f, indent=4)